In [51]:
import pandas as pd
import numpy as np

### Реализуем функции активации

In [52]:
def identity(data: np.ndarray) -> np.ndarray:
    return data


def identity_derivative(x: np.ndarray) -> np.ndarray:
    return np.ones_like(x)


def relu(x):
    return np.maximum(0, x)


def relu_derivative(x):
    return np.where(x > 0, 1, 0)


def tanh(data: np.ndarray) -> np.ndarray:
    return np.tanh(data)


def tanh_derivative(x: np.ndarray) -> np.ndarray:
    return 1.0 - tanh(x)**2

In [53]:
from typing import Callable

In [54]:
class ActivationFunction:
    def __init__(self
                 , name: str
                 , activation_function: Callable[[np.ndarray], np.ndarray]
                 , activation_derivative: Callable[[np.ndarray], np.ndarray]):
        self.__name = name
        self.__activation_function = activation_function
        self.__activation_derivative = activation_derivative

    def name(self):
        return self.__name

    def activate(self, X: np.ndarray):
        return self.__activation_function(X)

    def derivative(self, X: np.ndarray):
        return self.__activation_derivative(X)


activation_functions = [
    ActivationFunction("Identity", identity, identity_derivative),
    ActivationFunction("ReLU", relu, relu_derivative),
    ActivationFunction("Tanh", tanh, tanh_derivative)
]

In [55]:
EPS = 1e-15


def softmax(values: np.ndarray):
    max_value = np.max(values, axis=1, keepdims=True)
    exp_values = np.exp(values - max_value)
    exp_values_sum = np.sum(exp_values, axis=1, keepdims=True)
    return exp_values / exp_values_sum


def cross_entropy(y_pred: np.ndarray, y_true: np.ndarray):
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1 - eps)
    loss = -np.sum(y_true * np.log(y_pred))
    return loss / len(y_true)


def cross_entropy_gradient(y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1 - eps)
    y_true = y_true.reshape(-1, 1)
    return -(y_true / y_pred) + (1 - y_true) / (1 - y_pred)


In [56]:
class Layer:
    """
    Class which represents single layer of network

    It stores data about its weights' matrix, biases and activation function
    and can make forward pass and backpropagation
    """
    def  __init__(self
                 , input_size: int
                 , output_size: int
                 , activation_function: ActivationFunction):
        """
        Layer constructor
        Weights matrix shape will be input_size x output_size

        :param input_size: Size of input, which is size of previous layer output's size
        :param output_size: Output size
        :param activation_function: activation function
        """
        self.__input_size = input_size
        self.__output_size = output_size
        self.__activation_function = activation_function
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.biases = np.random.randn(1, output_size)
        self.Z = None
        self.A = None

    def forward(self, X: np.ndarray) -> np.ndarray:
        """
        Simple matrix transformation

        We multiply the matrix obtained as an input by the matrix of weights of this layer,
        make a shift and apply the activation function to this

        :param X: input matrix (result of previous layer or starting data)
        :return: Result of activation function applied to matrix transformation
        """
        self.Z = np.dot(X, self.weights) + self.biases
        self.A = self.__activation_function.activate(self.Z)
        return self.A


    def backward(self, dA: np.ndarray, learning_rate: float, X_prev: np.ndarray) -> np.ndarray:
        dZ = dA * self.__activation_function.derivative(self.Z)
        dW = np.dot(X_prev.T, dZ)
        db = np.sum(dZ, axis=0, keepdims=True)
        self.weights -= learning_rate * dW
        self.biases -= learning_rate * db
        dA_prev = np.dot(dZ, self.weights.T)
        return dA_prev



class Network:
    def __init__(self, layers: list[Layer], learning_rate: float, epochs: int):
        self.__layers = layers
        self.__learning_rate = learning_rate
        self.__epochs = epochs

    def train(self
              , X: np.ndarray
              , Y: np.ndarray) -> np.ndarray:

        for epoch in range(self.__epochs):
            A = X

            for layer in self.__layers:
                A = layer.forward(A)

            loss = cross_entropy(A, Y)
            dA = cross_entropy_gradient(A, Y)

            for i in reversed(range(len(self.__layers))):
                layer = self.__layers[i]
                if i == 0:
                    X_prev = X
                else:
                    X_prev = self.__layers[i-1].A

                dA = layer.backward(dA, self.__learning_rate, X_prev)


            if epoch % 100 == 0:
                print(f"Epoch: {epoch}, loss: {loss}")
        return A

### Теперь будем разбираться с тем, как заиспользовать полученную сеть для задачи классификации рейтингов вин

In [57]:
df = pd.read_csv("../../raw_dataframe.csv", encoding="cp1252")
df.head()

,name,price,rating,country,region,sweetness,grape,manufacturer,strength,volume
0,"Vino Tracer Riesling, Weinkellerei Hechtsheim,...",1312.0,4.7,Germanija,Pfal'ts,semi-dry,risling 100%,Weinkellerei Hechtsheim,12.0,0.75
1,"Vino Lighea, Donnafugata, 2021",4990.0,4.8,Italija,Sitsilija,dry,zibibbo 100%,Donnafugata,12.5,0.75
2,"Vino Chenin Blanc, David & Nadia, 2022",6790.0,4.5,Juzhnaja Afrika,Svortlend,dry,shenen blan 100%,David & Nadia,12.5,0.75
3,"Vino Pinot Noir Alpine Vineyard, Rhys Vineyard...",34990.0,5.0,Soedinennye Shtaty Ameriki,Kalifornija,dry,pino nuar,Rhys Vineyards,12.9,0.75
4,"Vino Grain de Gris, Listel, 2022",1393.0,4.6,Frantsija,Langedok-Russil'on,dry,sira,Listel,12.0,0.75


In [58]:
df.shape

(3550, 10)

In [59]:
df = df.dropna()

In [60]:
df['grape'].map(type).value_counts()

grape
<class 'str'>    3438
Name: count, dtype: int64

In [61]:
def process_grapes(grape: str) -> str:
    if grape.endswith("%"):
        return ' '.join(grape.split()[:-1])

    return grape


df["grape"] = df["grape"].map(process_grapes)
df.head()

,name,price,rating,country,region,sweetness,grape,manufacturer,strength,volume
0,"Vino Tracer Riesling, Weinkellerei Hechtsheim,...",1312.0,4.7,Germanija,Pfal'ts,semi-dry,risling,Weinkellerei Hechtsheim,12.0,0.75
1,"Vino Lighea, Donnafugata, 2021",4990.0,4.8,Italija,Sitsilija,dry,zibibbo,Donnafugata,12.5,0.75
2,"Vino Chenin Blanc, David & Nadia, 2022",6790.0,4.5,Juzhnaja Afrika,Svortlend,dry,shenen blan,David & Nadia,12.5,0.75
3,"Vino Pinot Noir Alpine Vineyard, Rhys Vineyard...",34990.0,5.0,Soedinennye Shtaty Ameriki,Kalifornija,dry,pino nuar,Rhys Vineyards,12.9,0.75
4,"Vino Grain de Gris, Listel, 2022",1393.0,4.6,Frantsija,Langedok-Russil'on,dry,sira,Listel,12.0,0.75


In [62]:
import category_encoders

encoder = category_encoders.BaseNEncoder(base = 4, cols = ['country', 'region', 'grape', 'manufacturer']).fit(df)
df = encoder.transform(df)
df.head()

,name,price,rating,country_0,country_1,country_2,region_0,region_1,region_2,region_3,...,grape_1,grape_2,grape_3,manufacturer_0,manufacturer_1,manufacturer_2,manufacturer_3,manufacturer_4,strength,volume
0,"Vino Tracer Riesling, Weinkellerei Hechtsheim,...",1312.0,4.7,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,1,12.0,0.75
1,"Vino Lighea, Donnafugata, 2021",4990.0,4.8,0,0,2,0,0,0,2,...,0,0,2,0,0,0,0,2,12.5,0.75
2,"Vino Chenin Blanc, David & Nadia, 2022",6790.0,4.5,0,0,3,0,0,0,3,...,0,0,3,0,0,0,0,3,12.5,0.75
3,"Vino Pinot Noir Alpine Vineyard, Rhys Vineyard...",34990.0,5.0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,1,0,12.9,0.75
4,"Vino Grain de Gris, Listel, 2022",1393.0,4.6,0,1,1,0,0,1,1,...,0,1,1,0,0,0,1,1,12.0,0.75


In [63]:
for column in ["price", "strength", "volume"]:
    df[column] = (df[column] - df[column].mean()) / df[column].std()

df.head(10)

,name,price,rating,country_0,country_1,country_2,region_0,region_1,region_2,region_3,...,grape_1,grape_2,grape_3,manufacturer_0,manufacturer_1,manufacturer_2,manufacturer_3,manufacturer_4,strength,volume
0,"Vino Tracer Riesling, Weinkellerei Hechtsheim,...",-0.229692,4.7,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,1,-1.133567,-0.121366
1,"Vino Lighea, Donnafugata, 2021",-0.180184,4.8,0,0,2,0,0,0,2,...,0,0,2,0,0,0,0,2,-0.695962,-0.121366
2,"Vino Chenin Blanc, David & Nadia, 2022",-0.155955,4.5,0,0,3,0,0,0,3,...,0,0,3,0,0,0,0,3,-0.695962,-0.121366
3,"Vino Pinot Noir Alpine Vineyard, Rhys Vineyard...",0.223636,5.0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,1,0,-0.345878,-0.121366
4,"Vino Grain de Gris, Listel, 2022",-0.228602,4.6,0,1,1,0,0,1,1,...,0,1,1,0,0,0,1,1,-1.133567,-0.121366
5,"Vino Volnay Premier Cru Clos des Chenes, Domai...",0.068839,5.0,0,1,1,0,0,1,2,...,0,1,0,0,0,0,1,2,0.616852,-0.121366
6,"Vino Tenuta Regaleali Cygnus, Tasca d'Almerita...",-0.186914,4.5,0,0,2,0,0,0,2,...,0,1,2,0,0,0,1,3,0.616852,-0.121366
7,"Vino Bordeaux des Iles, Domaine de l'Ile Marga...",-0.173454,5.0,0,1,1,0,0,1,3,...,0,1,3,0,0,0,2,0,0.179247,-0.121366
8,"Vino Riesling Kastelberg Grand Cru Le Chateau,...",-0.005195,5.0,0,1,1,0,0,2,0,...,0,0,1,0,0,0,2,1,-0.258357,-0.121366
9,"Vino Primofiore, Giuseppe Quintarelli, 2021, 1...",0.290940,5.0,0,0,2,0,0,2,1,...,0,2,0,0,0,0,2,2,0.616852,2.631291


In [64]:
sweetness_types = { sweet: idx for (idx, sweet) in enumerate(list(df['sweetness'].unique())) }
sweetness_types

{'semi-dry': 0, 'dry': 1, 'sweet': 2, 'semi-sweet': 3}

In [65]:
df['sweetness'] = df['sweetness'].map(lambda x: sweetness_types[x])
df.head()

,name,price,rating,country_0,country_1,country_2,region_0,region_1,region_2,region_3,...,grape_1,grape_2,grape_3,manufacturer_0,manufacturer_1,manufacturer_2,manufacturer_3,manufacturer_4,strength,volume
0,"Vino Tracer Riesling, Weinkellerei Hechtsheim,...",-0.229692,4.7,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,1,-1.133567,-0.121366
1,"Vino Lighea, Donnafugata, 2021",-0.180184,4.8,0,0,2,0,0,0,2,...,0,0,2,0,0,0,0,2,-0.695962,-0.121366
2,"Vino Chenin Blanc, David & Nadia, 2022",-0.155955,4.5,0,0,3,0,0,0,3,...,0,0,3,0,0,0,0,3,-0.695962,-0.121366
3,"Vino Pinot Noir Alpine Vineyard, Rhys Vineyard...",0.223636,5.0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,1,0,-0.345878,-0.121366
4,"Vino Grain de Gris, Listel, 2022",-0.228602,4.6,0,1,1,0,0,1,1,...,0,1,1,0,0,0,1,1,-1.133567,-0.121366


In [66]:
nbins = 5
df['rating'], bins = pd.qcut(df['rating'], q=nbins, labels=False, retbins=True)

bins

array([0.  , 4.  , 4.48, 4.6 , 4.9 , 5.  ])

In [67]:
df['rating'].value_counts()

rating
0    773
2    733
3    732
1    602
4    598
Name: count, dtype: int64

In [68]:
df.head()

,name,price,rating,country_0,country_1,country_2,region_0,region_1,region_2,region_3,...,grape_1,grape_2,grape_3,manufacturer_0,manufacturer_1,manufacturer_2,manufacturer_3,manufacturer_4,strength,volume
0,"Vino Tracer Riesling, Weinkellerei Hechtsheim,...",-0.229692,3,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,1,-1.133567,-0.121366
1,"Vino Lighea, Donnafugata, 2021",-0.180184,3,0,0,2,0,0,0,2,...,0,0,2,0,0,0,0,2,-0.695962,-0.121366
2,"Vino Chenin Blanc, David & Nadia, 2022",-0.155955,2,0,0,3,0,0,0,3,...,0,0,3,0,0,0,0,3,-0.695962,-0.121366
3,"Vino Pinot Noir Alpine Vineyard, Rhys Vineyard...",0.223636,4,0,1,0,0,0,1,0,...,0,1,0,0,0,0,1,0,-0.345878,-0.121366
4,"Vino Grain de Gris, Listel, 2022",-0.228602,2,0,1,1,0,0,1,1,...,0,1,1,0,0,0,1,1,-1.133567,-0.121366


In [69]:
from sklearn.model_selection import train_test_split


X = df.drop(["name", "rating"], axis=1)
y = df["rating"]

In [70]:
X.head()

,price,country_0,country_1,country_2,region_0,region_1,region_2,region_3,sweetness,grape_0,grape_1,grape_2,grape_3,manufacturer_0,manufacturer_1,manufacturer_2,manufacturer_3,manufacturer_4,strength,volume
0,-0.229692,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,-1.133567,-0.121366
1,-0.180184,0,0,2,0,0,0,2,1,0,0,0,2,0,0,0,0,2,-0.695962,-0.121366
2,-0.155955,0,0,3,0,0,0,3,1,0,0,0,3,0,0,0,0,3,-0.695962,-0.121366
3,0.223636,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,-0.345878,-0.121366
4,-0.228602,0,1,1,0,0,1,1,1,0,0,1,1,0,0,0,1,1,-1.133567,-0.121366


In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
for f in activation_functions:
    print(f.name())

Identity
ReLU
Tanh


In [72]:
X_train.shape

(2750, 20)

In [73]:
layers = [
    Layer(X_train.shape[1], 15, activation_functions[2]),
    Layer(15, 10, activation_functions[1]),
    Layer(10, 5, activation_functions[2]),
    Layer(5, 1, activation_functions[1])
]

network = Network(layers, 0.15, 1000)
y_pred = network.train(X_train.to_numpy(), y_train.to_numpy())


Epoch: 0, loss: 182779.20468186727
Epoch: 100, loss: 182779.20468186727
Epoch: 200, loss: 182779.20468186727
Epoch: 300, loss: 182779.20468186727
Epoch: 400, loss: 182779.20468186727
Epoch: 500, loss: 182779.20468186727
Epoch: 600, loss: 182779.20468186727
Epoch: 700, loss: 182779.20468186727
Epoch: 800, loss: 182779.20468186727
Epoch: 900, loss: 182779.20468186727


In [74]:
print(y_pred)

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [76]:
from sklearn.metrics import f1_score

score = f1_score(y_train, y_pred, average="micro")
print(f"F-score: {score}")

F-score: 0.228
